In [1]:
import os 


In [2]:
%pwd

'd:\\voting_application\\Text_summariser-NLP-\\Text_summariser-NLP-\\research'

In [3]:
os.chdir("../")

In [31]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class ModelTrainerConfig:
    
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int
    


In [46]:
from text_summarizer.constants import *
from text_summarizer.utlis.common import  read_yaml, create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config (self) -> ModelTrainerConfig:
        


        config = self.config.model_trainer
        params =self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
              root_dir = self.config.artifacts_root,
              data_path = self.config.data_path,
              model_ckpt = self.config.model_ckpt,
              num_train_epochs = params.num_train_epochs,
              warmup_steps = params.warmup_steps,
              per_device_train_batch_size = params.per_device_train_batch_size,
              weight_decay = params.weight_decay,
              logging_steps = params.logging_steps,
              eval_steps = params.eval_steps,
              save_steps = params.save_steps,
              gradient_accumulation_steps = params.gradient_accumulation_steps,
          )

        return model_trainer_config

    

In [39]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch


In [40]:


trainer_args = TrainingArguments(
    output_dir = 'pegasus -samsum', num_train_epochs = 1 , warmup_steps = 500,
    per_device_train_batch_size = 8, weight_decay = 0.01, logging_steps = 100,
    eval_steps = 100, save_steps = 100, gradient_accumulation_steps = 16)

In [44]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.artifacts_root ,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["test"],
            eval_dataset=dataset_samsum_pt["validation"]
        )

        trainer.train()

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

In [47]:
config_manager = ConfigurationManager()
model_trainer_config = config_manager.get_model_trainer_config()
trainer = ModelTrainer(model_trainer_config)
trainer.train()

[2024-07-02 00:14:40,652: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-02 00:14:40,661: INFO: common: yaml file: paramas.yaml loaded successfully]
[2024-07-02 00:14:40,667: INFO: common: created directory at: artifacts]
[2024-07-02 00:14:40,672: INFO: common: created directory at: artifacts/model_trainer]


BoxKeyError: "'ConfigBox' object has no attribute 'data_path'"